In [2]:
import sys
sys.path[0] = ('/home/labs/waic/omrik/DNN-Challenge')
from fastai.vision import *
import pre
import resample
from metrics import Pearson
from fastai.utils.mod_display import *
from utils import ProgressBarCtx

root = Path('/home/labs/waic/omrik/DNN-Challenge/data').resolve()
train = root / 'train'
val = root / 'val'

In [ ]:
def normalize_time(series):
    # 1440 minutes in a day
    normalized = (series.hour * 60 + series.minute) / 1440
    return normalized

def get_data(data_dir):
    cgm, meals = pre.get_dfs(data_dir)
    meals = resample.resample_meals(cgm, meals, 15)
    meals = pd.concat((meals, cgm), axis=1)
    meals['time'] = normalize_time(meals.index.get_level_values('Date'))
    cgm, y = pre.build_cgm(cgm, drop=False)
    return cgm, meals, y

class ContData(Dataset):
    def __init__(self, cgm, meals, y):
        self.cgm = cgm.loc[y.index].dropna(how='any', axis=0)
        self.meals = meals
        self.y = y
        
    def __len__(self):
        return len(self.cgm)
    
    def __getitem__(self, i):
        index = self.meals.index.get_loc(self.cgm.index[i])
        values = self.meals[index-48:index+1].values
        index = self.y.index.get_loc(self.cgm.index[i])
        target = self.y.iloc[index-48:index+1]
        x, y = torch.tensor(values, dtype=torch.float), torch.tensor(target.values, dtype=torch.float)
        return x, y
    
def loss(outputs, targets, start_idx=0):
    targets = targets.transpose(0, 1)
    return F.mse_loss(outputs[start_idx:], targets[start_idx:])

In [ ]:
train_data = get_data(train)
val_data = get_data(val)

In [ ]:
train_ds = ContData(*train_data)
val_ds = ContData(*val_data)
data = DataBunch.create(train_ds, val_ds, bs=512)
data

In [ ]:
class WindowPearson(Pearson):
    def __init__(self, gt):
        super().__init__(gt)
        
    def on_batch_end(self, last_output, last_target, **kwargs):
        super().on_batch_end(last_output[-1], last_target, **kwargs)
        
p = WindowPearson(val_ds.y.loc[val_ds.cgm.index].dropna(how='any', axis=0))

In [ ]:
MAX_LENGTH = 49


class Seq2Lin(Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0):
        super().__init__()
        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, 8)

        
    def forward(self, input):
        input = input.transpose(0, 1)
        encoder_outputs, _ = self.encoder(input)
        out = self.decoder(encoder_outputs)
        return out

In [ ]:
dropout = 0.1
start_idx = -11

In [ ]:
class AddNoise(Callback):
    
    def __init__(self, std):
        self.std = std
        
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        if not train:
            return
        
        size = last_input.shape[:2]
        noise = torch.normal(0, self.std, size).to(last_input.device)
        last_input[..., -2] += noise
        return {'last_input': last_input}


In [ ]:
model = Seq2Lin(38, 128, num_layers=4, dropout=dropout)
learner = Learner(data, model, loss_func=partial(loss, start_idx=start_idx), metrics=p)

In [ ]:
with ProgressBarCtx(learner, False) as l:
    l.lr_find()
    l.recorder.plot(suggestion=True)

In [ ]:
with ProgressBarCtx(learner, False) as l:
    l.fit_one_cycle(10, 5e-4)